L'objectif ici est d'évaluer les réponses des modèles de langage (LLM) en s'appuyant sur une métrique clairement définie. Le but est d'automatiser des tests basés sur cette métrique afin d'éviter une approche empirique et subjective.

L'automatisation des tests nécessite un travail de développement important, c'est pourquoi l'objectif est d'examiner comment `langSmith` peut nous aider à réduire ces efforts.
NB : `LangSmith` est une plateforme permettant de créer des applications LLM de niveau production. Suivez et évaluez votre candidature pour une livraison rapide et sereine:
https://docs.langchain.com/langsmith/home


In [ ]:
# install and import needed dependencies

!pip install pandas matplotlib seaborn langchain-mistralai langchain-huggingface langsmith scikit-learn numpy

import os
from datetime import datetime
from langchain_mistralai import ChatMistralAI
from langchain_huggingface import HuggingFaceEmbeddings
from langsmith import Client
from langsmith.evaluation import evaluate
from langsmith.schemas import Example, Run
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [7]:
# Configuration LangSmith
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "performance-analysis"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Please enter your langSmith API key (hit enter): ")

In [8]:
# Set up models
MISTRAL_APIKEY = getpass.getpass("Please enter your Mistral API key (hit enter): ")

client = Client()
llm = ChatMistralAI(model="mistral-small-latest", mistral_api_key=MISTRAL_APIKEY)
embeddings_model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

In [10]:
dataset_name = "math-qa-performance"

def create_large_dataset():
    """create data set input/output(expected)"""

    try:
        dataset = client.create_dataset(dataset_name)
    except:
        # If dataset already exist
        dataset = client.read_dataset(dataset_name=dataset_name)

    examples = [
        {"inputs": {"question": "Combien font deux plus trois?", "difficulty": "easy"},
         "outputs": {"expected": "Deux plus trois égale cinq."}},
        {"inputs": {"question": "Quelle est la différence entre dix et quatre?", "difficulty": "easy"},
         "outputs": {"expected": "La différence entre dix et quatre est six."}},
        {"inputs": {"question": "Quel est le résultat de cinq multiplié par deux?", "difficulty": "easy"},
         "outputs": {"expected": "Cinq multiplié par deux donne dix."}},

        {"inputs": {"question": "Comment calculer douze fois quinze?", "difficulty": "medium"},
         "outputs": {"expected": "Pour calculer douze fois quinze, on multiplie 12 par 15, ce qui donne 180."}},
        {"inputs": {"question": "Quelle est la racine carrée de soixante-quatre?", "difficulty": "medium"},
         "outputs": {"expected": "La racine carrée de soixante-quatre est huit, car huit au carré égale soixante-quatre."}},
        {"inputs": {"question": "Comment diviser cent quarante-quatre par douze?", "difficulty": "medium"},
         "outputs": {"expected": "Pour diviser cent quarante-quatre par douze, on obtient douze comme résultat."}},

        {"inputs": {"question": "Comment résoudre l'équation deux x plus cinq égale quinze?", "difficulty": "hard"},
         "outputs": {"expected": "Pour résoudre 2x + 5 = 15, on soustrait 5 des deux côtés pour obtenir 2x = 10, puis on divise par 2 pour trouver x = 5."}},
        {"inputs": {"question": "Quelle est la valeur du sinus de trente degrés?", "difficulty": "hard"},
         "outputs": {"expected": "Le sinus de trente degrés est égal à un demi ou 0,5."}},
        {"inputs": {"question": "Comment calculer la dérivée de x au carré plus trois x?", "difficulty": "hard"},
         "outputs": {"expected": "La dérivée de x² + 3x est 2x + 3, en appliquant la règle de dérivation des polynômes."}},
    ]

    for example in examples:
        client.create_example(
            inputs=example["inputs"],
            outputs=example["outputs"],
            dataset_id=dataset.id
        )

In [11]:
def math_qa_chain(inputs):
    """"""
    question = inputs["question"]
    difficulty = inputs.get("difficulty", "unknown")

    if difficulty == "hard":
        prompt = f"Tu es un professeur de mathématiques. Explique clairement et en détail comment résoudre cette question: {question}. Donne une réponse complète en français."
    else:
        prompt = f"Réponds à cette question mathématique en expliquant ta réponse en français de manière claire et naturelle: {question}"

    response = llm.invoke(prompt)
    return {"answer": response.content, "difficulty": difficulty}

In [12]:
def semantic_similarity_evaluator(run: Run, example: Example) -> dict:
    """"""
    predicted_text = run.outputs.get("answer", "")
    expected_text = example.outputs.get("expected", "")

    if not predicted_text or not expected_text:
        return {"key": "semantic_similarity", "score": 0}

    try:

        predicted_embedding = embeddings_model.embed_query(predicted_text)
        expected_embedding = embeddings_model.embed_query(expected_text)

        similarity = cosine_similarity(
            np.array(predicted_embedding).reshape(1, -1),
            np.array(expected_embedding).reshape(1, -1)
        )[0][0]

        return {
            "key": "semantic_similarity",
            "score": float(similarity),
            "comment": f"Similarité: {similarity:.3f}"
        }
    except Exception as e:
        return {
            "key": "semantic_similarity",
            "score": 0,
            "comment": f"Erreur: {str(e)}"
        }

In [13]:
def run_comprehensive_evaluation():
    """Lance une évaluation complète et retourne les données brutes - Focus semantic similarity"""

    print("🚀 Running...")

    for run_number in range(3):  # 3 exécutions pour avoir des données
        print(f"   Run {run_number + 1}/3...")

        evaluate(
            math_qa_chain,
            data=dataset_name,
            evaluators=[
                semantic_similarity_evaluator
            ],
            experiment_prefix=f"math-performance-run-{run_number}",
            metadata={
                "run_number": run_number,
                "timestamp": datetime.now().isoformat()
            }
        )

In [15]:
def run_complete_analysis():
    """Workflow complet: Tests LangSmith → Données → Visualisations"""

    print("🔬 DÉMARRAGE DE L'ANALYSE COMPLÈTE")
    print("=" * 50)

    # 1. Créer le dataset
    create_large_dataset()
    print("✅ Dataset créé")

    run_comprehensive_evaluation()

# Exécution
if __name__ == "__main__":
    run_complete_analysis()

🔬 DÉMARRAGE DE L'ANALYSE COMPLÈTE
✅ Dataset créé
🚀 Running...
   Run 1/3...
View the evaluation results for experiment: 'math-performance-run-0-edd9df7c' at:
https://smith.langchain.com/o/1f3656ac-a27f-4b91-be8d-ff9ac7fdf742/datasets/e1ee4503-359e-4033-9199-3dd06a6781fe/compare?selectedSessions=cce86fa6-83cb-4509-9cdc-085171bb9de9




9it [00:38,  4.23s/it]


   Run 2/3...
View the evaluation results for experiment: 'math-performance-run-1-c9095006' at:
https://smith.langchain.com/o/1f3656ac-a27f-4b91-be8d-ff9ac7fdf742/datasets/e1ee4503-359e-4033-9199-3dd06a6781fe/compare?selectedSessions=2951ae3f-b025-4795-bf2d-4aafad07a993




9it [00:34,  3.81s/it]


   Run 3/3...
View the evaluation results for experiment: 'math-performance-run-2-91fa9207' at:
https://smith.langchain.com/o/1f3656ac-a27f-4b91-be8d-ff9ac7fdf742/datasets/e1ee4503-359e-4033-9199-3dd06a6781fe/compare?selectedSessions=fea159d5-7ed8-4744-b75a-8b95374b9d58




9it [00:37,  4.22s/it]
